<a href="https://colab.research.google.com/github/polymathkids/AdvPredModels/blob/main/HW7Q2_Staff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q2 HW7 AMP Staff Graded:

Consider the following data sets:

TexasCityDistanceMatrix.csv
TexasCityKey.csv

This data is a subset of the National Bureau of Economic Research's City Distances database. The TexasCityDistanceMatrix.csv file contains a sparse, square matrix, in which the cells represent the distance between the cities on the respective row and column. Each row and column represents an individual city and their distance to the cities on the corresponding row/columns. The TexasCityKey.csv file contains a key for the id's in the other file, connecting the IDs to the names of the cities. Note there are repeated city names in this file. This is not an error nor does it require any data cleaning, as some cities are not uniquely named. (Hint: When you load the TexasCityDistanceMatrix.csv file, indicate the first column as indices.)

In [2]:
import pandas as pd
import numpy as np

# read in the CSV file as a DataFrame
DistMatrix = pd.read_csv('/content/drive/MyDrive/AMP/TexasCityDistanceMatrix.csv', index_col = 0)
TexasKey = pd.read_csv('/content/drive/MyDrive/AMP/TexasKey.csv')

# convert the DataFrame to a matrix
CityDistMatrix = DistMatrix.values

a. What is the maximum value in this matrix? What does this imply about the missing values of the matrix?

In [3]:
# find the maximum value of the matrix
max_dist = np.max(CityDistMatrix)

# print the maximum value
print(max_dist)

# ignore nan values
max_dist = np.nanmax(CityDistMatrix)
print(max_dist)

nan
99.99971353


The maximum value in the matrix initially returned an nan value but if nan values are ignored, the max value returned in 99.999. This tells me that the matrix only defines distances between cities that are less than 100 and nan values are likely those values that are greater than 100 miles apart.


b. What is the closest location to Katy city"? How many locations are within 25 miles of Flower Mound town" (excluding the town itself)?

In [4]:
#Need to remove 0's in dist matrix or closet location will always be itself
DistMatrix_z = DistMatrix.replace(0, np.nan)

# find the row in Texas Key that equals 'Katy city'
matching = TexasKey.loc[TexasKey['Name'] == 'Katy city']
#store the value of the Place column as the index we are looking for
city_index = matching['Place']

#grab the row for Katy city using the saved index value
min2Katy = DistMatrix_z.iloc[city_index,:].idxmin(axis = 1, skipna = True)
#get just the value of the place and strip away the brackets and V prefix
min2Katy = str(min2Katy.values)
min2Katy = int(min2Katy[3:-2])
#use result to find the matching name in TexasKey
print(TexasKey[TexasKey['Place'] == min2Katy])


      Place            Name
1128   1129  Oak Grove town


The location closest to Katy city is Oak Grove town.


In [5]:
# find the row in Texas Key that equals 'Flower Mound town'
matching = TexasKey.loc[TexasKey['Name'] == 'Flower Mound town']
#store the value of the Place column as the index we are looking for
city_index = matching['Place']

#save the values  in the row for Flower Mound town
#using the saved index value that are less than or equal to 25
#use DistMatrix_z that excludes the 0 values
FlowerMound = DistMatrix_z.iloc[city_index,:][DistMatrix_z.iloc[city_index,:]<=25].dropna(axis =1)
print(len(FlowerMound.columns)) #returns number of towns within 25 miles
FlowerMound = list(FlowerMound.columns) #grab col names and save as list
for i in range(len(FlowerMound)):
    FlowerMound[i] = FlowerMound[i][1:] #remove V prefix
    #FlowerMound is now saved as searchable locations for towns w/i 25 mi of FlowerMound


5


There are 5 locations within 25 miles of Flower Mound, excluding the town of Flower Mound itself.


c. Find the time it takes to sum all of the elements in the matrix.

In [6]:
import time
start = time.time()
np.sum(CityDistMatrix)
end = time.time()
print(end-start)

0.002972126007080078


d. Suppose you are interested in examining only the relationships between cities that are relatively close to one another. For all distances greater than 50 miles apart, set the value to zero. Additionally, set the missing values equal to zero. Compute the time it takes to sum all the elements in this new matrix. How does this compare to your previous results?

In [7]:
#replace values > 50 with 0
CityDistMatrix[CityDistMatrix>50] = 0

#replace nan values with 0
CityDistMatrix = np.nan_to_num(CityDistMatrix, nan = 0)

start = time.time()
np.sum(CityDistMatrix)
end = time.time()
print(end-start)


0.0029838085174560547


The original sum took approximately 0.006 seconds. The reduced and cleaned matrix took two-thirds of that time or approximately 0.004 seconds.

e. Now consider taking advantage of the missing data in this matrix by storing it as a sparse matrix. Using python's scipy.sparse package, use the csr_matrix function to store the original matrix (without removing distances greater than 50 miles apart) as a sparse matrix. Ensure that the missing values are not stored but all other values (including zeros) are kept. Compute the time it takes to sum all the elements in this new sparse matrix. How does this compare to your previous results?

In [10]:
from scipy.sparse import csr_matrix
# revert to original matrix by converting the orig DataFrame to a matrix
CityDistMatrix = DistMatrix.values
data_size = CityDistMatrix.nbytes/(1024**2)
print('Original size of matrix: '+ '%3.2f' %data_size + ' MB')

#print(CityDistMatrix.shape)
unsparseTexas = csr_matrix(CityDistMatrix)
data_size = unsparseTexas.data.size/(1024**2)
print('Size of full csr matrix with zeros and NaNs: '+ '%3.2f' %data_size + ' MB')

#replace values = 0 with -1
CityDistMatrix[CityDistMatrix==0] = -1

#remove nan values
CityDistMatrix = CityDistMatrix[~np.isnan(CityDistMatrix)]

sparseTexas = csr_matrix(CityDistMatrix)
data_size = sparseTexas.data.size/(1024**2)
print('Size of matrix w/o NaNs: '+ '%3.2f' %data_size + ' MB')

#replace values = -1 with 0
sparseTexas = sparseTexas.toarray() #switch to dense array
sparseTexas[sparseTexas==-1] = 0
sparseTexas = csr_matrix(sparseTexas)
data_size = sparseTexas.data.size/(1024**2)
print('Size of matrix w/ explicit zeros: '+ '%3.2f' %data_size + ' MB')

start = time.time()
print("Sum is: ", sparseTexas.sum(axis=None))
end = time.time()
print("It took ", end-start, " seconds")

Original size of matrix: 23.42 MB
Size of full csr matrix with zeros and NaNs: 2.93 MB
Size of matrix w/o NaNs: 0.39 MB
Size of matrix w/ explicit zeros: 0.15 MB
Sum is:  4504877.859086584
It took  0.002202749252319336  seconds



f. Knowing that the matrix is symmetric and has zeros along the diagonal, how else could you save storage space? 

Because the matrix is symmetric, you could also just store the lower or upper triangle of the matrix which would retain only the elements below or above the main diagonal of the Texas City Distance matrix. You could retain or eliminate the diagonal in retention depending on whether you needed that information. In this case, city=city on the diagonal so I assume you would eliminate it to save additional space. In Python, you could store the lower triangle of a matrix with the `np.tril()` function. The `k` parameter of that function indicates whether or not you want to retain the diagonal. This would require additional processing to find matches and distances between cities.

Another  option would be to use the eliminate_zeros() and sum_duplicates() function of csr_matrix to remove zero values and then eliminate any duplicate enties by summing them together. As in-place operations, these functions can further reduce the size needed to store the matrix without data loss.

In [14]:
CityDistMatrix = DistMatrix.values
HalfCity = np.tril(CityDistMatrix, k=-1)
#replace values = 0 with -1
HalfCity[HalfCity==0] = -1

#remove nan values
HalfCity = HalfCity[~np.isnan(HalfCity)]
HalfTexas = csr_matrix(HalfCity)
HalfTexas = HalfTexas.toarray() #switch to dense array
HalfTexas[HalfTexas==-1] = 0
HalfTexas = csr_matrix(HalfTexas)
data_size = HalfTexas.data.size/(1024**2)
print('Size of matrix- using only lower triangle: '+ '%3.2f' %data_size + ' MB')

start = time.time()
print("Sum is: ", HalfTexas.sum(axis=None))
end = time.time()
print("It took ", end-start, " seconds")
print("")
#replace values = 0 with -1
CityDistMatrix[CityDistMatrix==0] = -1

#remove nan values
CityDistMatrix = CityDistMatrix[~np.isnan(CityDistMatrix)]
Texas = csr_matrix(CityDistMatrix)
Texas = Texas.toarray() #switch to dense array
Texas[Texas==-1] = 0
Texas = csr_matrix(Texas)

Texas.eliminate_zeros()
Texas.sum_duplicates()
data_size = Texas.data.size/(1024**2)
print('Size of matrix- after using sum_duplicates: '+ '%3.2f' %data_size + ' MB')

start = time.time()
print("Sum is: ", Texas.sum(axis=None))
end = time.time()
print("It took ", end-start, " seconds")


Size of matrix- using only lower triangle: 0.07 MB
Sum is:  2252438.929543232
It took  0.003416299819946289  seconds

Size of matrix- after using sum_duplicates: 0.15 MB
Sum is:  4504877.859086584
It took  0.001878499984741211  seconds
